<a href="https://colab.research.google.com/github/Vinicius-Tavares-Silva/Job-AI-Match/blob/main/Job_AI_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação dos módulos (Caso necessário)

In [ ]:
pip install pdfx
pip install requests==2.22.0 beautifulsoup4==4.8.1

Job AI Match: Analise seu curriculo com sua vaga dos sonhos, entenda seus pontos fortes e a melhorar!

In [59]:
# Importação dos módulos
import hashlib
import string
import requests
import textwrap
import pdfx
from pathlib import Path
from bs4 import BeautifulSoup
from IPython.display import display
from IPython.display import Markdown
from google.colab import files
from google.colab import userdata
import google.generativeai as genai

# Uso do secret do Colab
api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

# Setup do modelo
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]
system_instruction = "Voce é um recrutador da area de tecnologia, deve estruturar sua resposta sempre com 4 tópicos: Pontos Fortes, Pontos a Melhorar, Recomendações, Conclusão. Caso não identifique o curriculo ou os requisitos da vaga deve alertar o usuario"
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

# Definição dos metodos que serão utilizados

# Responsável por ler o arquivo pdf
def file_reader(file_name):
  try:
    pdf = pdfx.PDFx(file_name)
    return pdf.get_text()
  except Exception as e:
    print(f"Erro: Arquivo '{file_name}' fora do formato PDF")

# Faz o upload do arquivo
def file_upload():
  try:
    uploaded = files.upload()
    return list(uploaded.keys())[0]
  except Exception as e:
    print(f"Erro: Arquivo inválido")

# Formata texto pra markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Obtem as informações da vaga atraves de um web scraping
def get_job_info(job_url):
  try:
    html_text = requests.get(job_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    html_plain_text = soup.get_text().translate({ord(c): None for c in string.whitespace})
    return html_plain_text
  except Exception as e:
    print(f"Erro: Não foi possivel obter a vaga")

# Inicio da execução

print("Faça o upload do seu Currículo! (formato .pdf)")
# Campo para upload do arquivo
file_name = file_upload()
if file_name != None:
  file_content = file_reader(file_name)
  if file_content != None:
    # Instância do modelo
    recruiter = model.start_chat(history=[])

    # Input do link da vaga pelo usúario (Para maiores informações consulte o readme)
    print("\n")
    print("Cole o link da vaga a qual pretende aplicar (LinkedIn, Gupy...): ")
    prompt = input("")
    if len(prompt) > 0:
      job_info = get_job_info(prompt)
      if job_info != None:
        # Envio do currículo para o modelo
        recruiter.send_message("Curriculo: " + file_content)
        # Resposta do modelo com o Match da vaga
        response = recruiter.send_message("Com base no curriculo enviado, quais são os pontos fortes e quais os pontos a se melhorar em relação a vaga a seguir: " + job_info)
        display(to_markdown(f'{response.text}'))

Faça o upload do seu Currículo! (formato .pdf)


Saving Profile_linkedin.pdf to Profile_linkedin (11).pdf


Cole o link da vaga a qual pretende aplicar (LinkedIn, Gupy...): 
https://www.linkedin.com/jobs/view/3908755384


> ## Análise de Currículo para Vaga de Frontend Developer na Tata Consultancy Services
> 
> **Pontos Fortes:**
> 
> * **Experiência com Desenvolvimento Front-end:** O currículo de Vinícius demonstra experiência como desenvolvedor front-end, o que é um requisito fundamental para a vaga. Sua atuação na Omie e o conhecimento em Vue.js/React.js são pontos positivos.
> * **Conhecimento em JavaScript e TypeScript:** A vaga exige experiência comprovada em JavaScript e TypeScript, habilidades que Vinícius possui de acordo com seu currículo.
> * **Formação em Desenvolvimento Web:** A formação na Trybe demonstra o interesse e a dedicação de Vinícius em se especializar na área de desenvolvimento web, o que é valorizado pela empresa. 
> * **Experiência com Metodologias Ágeis:** A experiência anterior como analista de projetos sugere que Vinícius pode ter familiaridade com metodologias ágeis, um ponto positivo considerando o ambiente de trabalho da TCS.
> 
> **Pontos a Melhorar:**
> 
> * **Experiência com React e Remix:** A vaga menciona especificamente React e Remix como tecnologias desejáveis. Seria importante que Vinícius destacasse em seu currículo sua experiência com essas tecnologias ou, caso não as domine, demonstrasse interesse em aprendê-las.
> * **Experiência no Setor de Seguros:** A vaga menciona que o projeto é para uma seguradora global. Embora não seja um requisito obrigatório, conhecimento sobre o mercado de seguros poderia ser um diferencial. 
> * **Portfólio com Projetos Relevantes:**  É importante que Vinícius verifique se seu portfólio online inclui projetos que demonstrem suas habilidades em React, Remix e outras tecnologias relevantes para a vaga. 
> * **Tempo de Experiência:** A vaga exige mais de 3 anos de experiência como desenvolvedor front-end.  Embora Vinícius possua experiência na área, pode ser necessário destacar a relevância dos projetos e as tecnologias utilizadas para se equiparar aos demais candidatos.
> 
> **Recomendações:**
> 
> * **Destacar Experiência com React e Remix:** Se Vinícius possui experiência com React e Remix, deve evidenciá-la no currículo, descrevendo projetos ou atividades onde utilizou essas tecnologias.
> * **Aprender sobre o Mercado de Seguros:** Pesquisar sobre o mercado de seguros e as tecnologias utilizadas no setor pode ser um diferencial na entrevista.
> * **Reforçar o Portfólio:** Atualizar o portfólio com projetos que demonstrem habilidades em React, Remix e outras tecnologias relevantes para a vaga.
> * **Preparar-se para a Entrevista:** Pesquisar sobre a empresa, a vaga e o setor de seguros para demonstrar interesse e conhecimento na entrevista.
> 
> 
> **Conclusão:**
> 
> Vinícius possui um currículo com pontos fortes relevantes para a vaga de Frontend Developer na Tata Consultancy Services. Ao aprimorar alguns aspectos e se preparar para a entrevista, ele terá boas chances de se destacar no processo seletivo. 
